In [11]:
import pandas as pd # import for dataframe handle
import numpy as np # import for math and array operations
import matplotlib.pyplot as plt # import for visual representation
#import seaborn as sns # import for visual representation

import re
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup
import requests

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OrdinalEncoder,RobustScaler, StandardScaler, FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer, make_column_selector

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df = pd.read_csv("../raw_data/clean_df.csv")
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,soundtrack multiplayer singleplayer fast paced...
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,multiplayer shooter action online person team ...
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,tactic sci fi turn based mechs strategy turn b...
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,access simulation fps post apocalyptic surviva...
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,multiplayer rpg strategy play economy strategy...


In [63]:
def create_kmeans_pipeline(df):
    array_transf = FunctionTransformer(lambda array: array.toarray())
   
    desc_transf = make_pipeline(TfidfVectorizer(min_df = 0.04,ngram_range=(1,2)),array_transf, RobustScaler())
    
#     label_transf = FunctionTransformer(lambda model: pd.DataFrame(data=model.labels_, index=df.name.tolist(), columns = ['cluster']))


    preproc_basic = make_column_transformer(
        (desc_transf, 'game_description'),
        remainder='drop'
    )
    
    kmeans_pipe = make_pipeline(preproc_basic,KMeans(n_clusters=50))#,label_transf)
    return kmeans_pipe

In [64]:
kmeans_pipe=create_kmeans_pipeline(df)
kmeans_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('tfidfvectorizer',
                                                                   TfidfVectorizer(min_df=0.04,
                                                                                   ngram_range=(1,
                                                                                                2))),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(func=<function create_kmeans_pipeline.<locals>.<lambda> at 0x7fd17b0cfa60>)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  'game_description')])),
                ('kmeans', KMeans(n_clusters=50))])

In [71]:
df.shape

(24567, 14)

In [66]:
test=kmeans_pipe.fit_transform(df)

In [67]:
test.shape

(24567, 50)

In [70]:
test

array([[ 9.91829154,  6.46808844,  7.80900484, ..., 10.86538369,
        10.05929232,  9.61877992],
       [ 7.27297254,  6.30080913,  9.16651701, ..., 10.60289135,
         6.49887256,  8.0654901 ],
       [ 6.18411006,  7.08365666, 10.01097479, ...,  6.53581651,
         8.39933598,  6.20428185],
       ...,
       [ 4.8182103 ,  7.31836533, 11.15086483, ...,  9.34588776,
         8.10859694,  4.42264609],
       [ 5.77187951,  3.29989206,  9.81291477, ...,  7.13978238,
         6.80347825,  5.68500126],
       [ 9.53607632,  6.98345773,  6.04139653, ...,  8.46630373,
         9.67733121,  9.60966973]])

In [56]:
def create_knn_pipeline():
    array_transf = FunctionTransformer(lambda array: array.toarray())
    
    meta_transf = make_pipeline(TfidfVectorizer(min_df=0.05), array_transf,RobustScaler())
#     desc_transf = make_pipeline(TfidfVectorizer(min_df=0.1),array_transf, RobustScaler())
    
    ord_encoder = OrdinalEncoder(
        categories=[
            [
                "Overwhelmingly Negative",
                "Very Negative",
                "Negative",
                "Mostly Negative",
                'Mixed',
                "Mostly Positive",
                "Positive",
                "Very Positive",
                "Overwhelmingly Positive"
            ]],
        dtype=np.int64,
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )
    
    ord_transf = make_pipeline(ord_encoder, StandardScaler())
    
    num_transf = make_pipeline(StandardScaler())


    preproc_basic = make_column_transformer(
        (meta_transf, 'metadata'),
#         (desc_transf, 'game_description'),
        (ord_transf, ['reviews']),
        (num_transf, ['mature_content', 'achievements']),
        remainder='drop'
    )
    
    return preproc_basic

In [57]:
knn_pipe = create_knn_pipeline()
knn_pipe

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('tfidfvectorizer',
                                                  TfidfVectorizer(min_df=0.05)),
                                                 ('functiontransformer',
                                                  FunctionTransformer(func=<function create_knn_pipeline.<locals>.<lambda> at 0x7fd1aac063a0>)),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 'metadata'),
                                ('pipeline-2',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Ov...
                                                                              'Negative',
                                                                              'Mostly '
                                                                              'Negative',
                                                                              'Mixed',
                                                                              'Mostly '
                                                                              'Positive',
                                                                              'Positive',
                                                                              'Very '
                                                                              'Positive',
                                                                              'Overwhelmingly '
                                                                              'Positive']],
                                                                 dtype=<class 'numpy.int64'>,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['reviews']),
                                ('pipeline-3',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 ['mature_content', 'achievements'])])

In [58]:
parallel_pipe=make_union(knn_pipe,kmeans_pipe)
parallel_pipe

FeatureUnion(transformer_list=[('columntransformer',
                                ColumnTransformer(transformers=[('pipeline-1',
                                                                 Pipeline(steps=[('tfidfvectorizer',
                                                                                  TfidfVectorizer(min_df=0.05)),
                                                                                 ('functiontransformer',
                                                                                  FunctionTransformer(func=<function create_knn_pipeline.<locals>.<lambda> at 0x7fd1aac063a0>)),
                                                                                 ('robustscaler',
                                                                                  RobustScaler())]),
                                                                 'metadata'),
                                                                ('pipeline-2',
                                                                 Pipeline(step...
                                                                                                   TfidfVectorizer(min_df=0.04,
                                                                                                                   ngram_range=(1,
                                                                                                                                2))),
                                                                                                  ('functiontransformer',
                                                                                                   FunctionTransformer(func=<function create_kmeans_pipeline.<locals>.<lambda> at 0x7fd196503f70>)),
                                                                                                  ('robustscaler',
                                                                                                   RobustScaler())]),
                                                                                  'game_description')])),
                                                ('kmeans',
                                                 KMeans(n_clusters=50)),
                                                ('functiontransformer',
                                                 FunctionTransformer(func=<function create_kmeans_pipeline.<locals>.<lambda> at 0x7fd196503940>))]))])

In [59]:
def create_parallel_pipeline(df):
    knn_pipe = create_knn_pipeline()
    kmeans_pipe=create_kmeans_pipeline(df)
    parallel_pipe=make_union(knn_pipe,kmeans_pipe)
    
    return parallel_pipe

In [61]:
create_parallel_pipeline(df)

FeatureUnion(transformer_list=[('columntransformer',
                                ColumnTransformer(transformers=[('pipeline-1',
                                                                 Pipeline(steps=[('tfidfvectorizer',
                                                                                  TfidfVectorizer(min_df=0.05)),
                                                                                 ('functiontransformer',
                                                                                  FunctionTransformer(func=<function create_knn_pipeline.<locals>.<lambda> at 0x7fd196503310>)),
                                                                                 ('robustscaler',
                                                                                  RobustScaler())]),
                                                                 'metadata'),
                                                                ('pipeline-2',
                                                                 Pipeline(step...
                                                                                                   TfidfVectorizer(min_df=0.04,
                                                                                                                   ngram_range=(1,
                                                                                                                                2))),
                                                                                                  ('functiontransformer',
                                                                                                   FunctionTransformer(func=<function create_kmeans_pipeline.<locals>.<lambda> at 0x7fd1aa2f4550>)),
                                                                                                  ('robustscaler',
                                                                                                   RobustScaler())]),
                                                                                  'game_description')])),
                                                ('kmeans',
                                                 KMeans(n_clusters=50)),
                                                ('functiontransformer',
                                                 FunctionTransformer(func=<function create_kmeans_pipeline.<locals>.<lambda> at 0x7fd1aa2f41f0>))]))])

In [ ]:
knn_pipe = create_knn_pipeline()
kmeans_pipe=create_kmeans_pipeline(df)

df.assign()